<a href="https://colab.research.google.com/github/MarVinikoi/DL_NafldReserch/blob/main/ImageClassif_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tcn
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd "/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/"

/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage


In [ ]:
from tensorflow.python.keras import callbacks
from tcn import TCN, tcn_full_summary
from tensorflow.python.keras.models import Sequential
import cv2
import tensorflow as tf
import numpy as np
import glob, os
from sklearn.model_selection import train_test_split
import math
import random
from google.colab.patches import cv2_imshow
import pandas as pd



In [ ]:
%%python3 --version

cv2.__version__

'4.6.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Variáveis Globais

path_doentes = "images/doentes"
path_saudaveis = "images/nao_doentes"

In [ ]:
def main():
  #dataAugmentation(True) #Só executa com o parametro setado em True
  #paddingImages()


  #numImgs()
  #model = tf.keras.models.load_model("/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/model.21-0.71")
  loadData()

if __name__ == "__main__":
  main()

images/doentes/doente_18
image 0 = images/doentes/doente_18/IR_41319.jpg
image 2 = images/doentes/doente_18/IR_41323.jpg
image 4 = images/doentes/doente_18/IR_41327.jpg
images/doentes/doente_1
image 0 = images/doentes/doente_1/IR_41519.jpg
image 4 = images/doentes/doente_1/IR_41527.jpg
image 7 = images/doentes/doente_1/IR_41533.jpg
images/doentes/doente_2
image 0 = images/doentes/doente_2/IR_41355.jpg
image 2 = images/doentes/doente_2/IR_41359.jpg
image 5 = images/doentes/doente_2/IR_41367.jpg
images/doentes/doente_3
image 0 = images/doentes/doente_3/IR_41023.jpg
image 2 = images/doentes/doente_3/IR_41027.jpg
image 5 = images/doentes/doente_3/IR_41033.jpg
images/doentes/doente_4
image 0 = images/doentes/doente_4/IR_43059.jpg
image 2 = images/doentes/doente_4/IR_43063.jpg
image 5 = images/doentes/doente_4/IR_43069.jpg
images/doentes/doente_5
image 0 = images/doentes/doente_5/IR_41473.jpg
image 6 = images/doentes/doente_5/IR_41485.jpg
image 11 = images/doentes/doente_5/IR_41497.jpg
image

In [ ]:

def dataAugmentation(on = False):
  if on:
    
    #Acessa todos os pacientes dentro do diretório doentes
    pacientes = glob.glob(path_doentes + "/*")
    n_prox_paciente = 19 #Num do paciente da sequência (o próximo para o qual irá ser criado uma pasta)
    for paciente in pacientes:
      #Cria o caminho para o paciente
      #Cria uma lista contendo os caminhos para todas as imagens de um paciente
      paths_imgs = sorted(glob.glob(paciente + "/*.jpg"))
      #Cria as pastas de "novos pacientes" correspondentes ao Data Augmentation do paciente atual
      
      pacienteHFlip_path = path_doentes + "/doente_" + str(n_prox_paciente) + "Hflip"
      pacienteVFlip_path = path_doentes + "/doente_" + str(n_prox_paciente) + "VFlip"
      pacienteRotate_path = path_doentes + "/doente_" + str(n_prox_paciente) + "Rotated"
      n_prox_paciente += 1
      os.mkdir(pacienteHFlip_path)
      os.mkdir(pacienteVFlip_path)
      os.mkdir(pacienteRotate_path)
      for i, img_path in enumerate(paths_imgs):

        image = cv2.imread(img_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        vflip_image = cv2.flip(image, 0)
        hflip_image = cv2.flip(image, 1)
        rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        vflip_image_name = pacienteVFlip_path + "/" + str(i) + "Vflip.jpg" 
        hflip_image_name = pacienteHFlip_path + "/" + str(i) + "Hflip.jpg"
        rotate_image_name = pacienteRotate_path + "/" + str(i) + "Rot.jpg"
        cv2.imwrite(vflip_image_name, vflip_image)
        cv2.imwrite(hflip_image_name, hflip_image)
        cv2.imwrite(rotate_image_name, rotated_image)

    
    #Acessa todos os pacientes dentro do diretório de não doentes
    pacientes = glob.glob(path_saudaveis + "/*")
    n_prox_paciente = 23
    for paciente in pacientes:
      paths_imgs = sorted(glob.glob(paciente + "/*.jpg"))

      pacienteHFlip_path = path_saudaveis + "/nao_doente_" + str(n_prox_paciente) + "Hflip"
      pacienteVFlip_path = path_saudaveis + "/nao_doente_" + str(n_prox_paciente) + "VFlip"
      pacienteRotate_path = path_saudaveis + "/nao_doente_" + str(n_prox_paciente) + "Rotated"
      n_prox_paciente += 1
      os.mkdir(pacienteHFlip_path)
      os.mkdir(pacienteVFlip_path)
      os.mkdir(pacienteRotate_path)
      for i, img_path in enumerate(paths_imgs):
        image = cv2.imread(img_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        vflip_image = cv2.flip(image, 0)
        hflip_image = cv2.flip(image, 1)
        rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        vflip_image_name = pacienteVFlip_path + "/" + str(i) + "Vflip.jpg" 
        hflip_image_name = pacienteHFlip_path + "/" + str(i) + "Hflip.jpg"
        rotated_image_name = pacienteRotate_path + "/" + str(i) + "Rot.jpg"
        cv2.imwrite(vflip_image_name, vflip_image)
        cv2.imwrite(hflip_image_name, hflip_image)
        cv2.imwrite(rotated_image_name, rotated_image)




In [ ]:
def numImgs():
  cont_tot = 0
  cont_doentes = 0
  cont_saudaveis = 0
  cont_paciente = 0

  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for img_path in (paths_imgs):
      cont_tot += 1
      cont_doentes += 1
      cont_paciente += 1
    print(path_paciente + f": {cont_paciente}")
    cont_paciente = 0

  for paciente in os.listdir(path_saudaveis):
    path_paciente = os.path.join(path_saudaveis, paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      cont_tot += 1
      cont_saudaveis += 1
      cont_paciente += 1
    print(path_paciente + f": {cont_paciente}")
    cont_paciente = 0

  print(f"\nTotal de imagens na base: {cont_tot}")
  print(f"Total de imagens de pacientes doentes na base: {cont_doentes}")
  print(f"Total de imagens de pacientes saudáveis na base: {cont_saudaveis}")

  return (cont_tot, cont_doentes, cont_saudaveis)

In [ ]:

#Função encarregada de carregar os dados em memória e chamar a função para realização do treino.
def loadData(model = None):

  treino = []
  teste = []
  val = []

  labels_treino = []
  labels_teste = []
  labels_val = []


  vetor_imgs = []
  vetor_final_imgs_doentes = []
  vetor_final_imgs_saudaveis = []


  #Carregando os vetores de imagens e os transformando em séries temporais
  #Vetor de Doentes
  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    print(path_paciente)
    #for i, img in enumerate(os.listdir(os.path.join(path_doentes, paciente))):
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):
        image = cv2.imread(img_path)
        print(f"image {i} = {img_path}")
        r_img, g_img, b_img = cv2.split(image)
        
        r_img = np.array(r_img)
        r_img = np.ndarray.flatten(r_img)

        vetor_imgs.append(r_img)
    vetor_final_imgs_doentes.append((vetor_imgs[0], vetor_imgs[1], vetor_imgs[2]))
    vetor_imgs.clear()
  vetor_final_imgs_doentes = np.array(vetor_final_imgs_doentes)  
  print(vetor_final_imgs_doentes.shape)
  


  #Vetor de Saudáveis
  for paciente in os.listdir(path_saudaveis):
    path_paciente = os.path.join(path_saudaveis, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):
        image = cv2.imread(img_path)
        print(img_path)
        r_img, g_img, b_img = cv2.split(image)
        
        r_img = np.array(r_img)
        r_img = np.ndarray.flatten(r_img)
            
        vetor_imgs.append(r_img)
    vetor_final_imgs_saudaveis.append((vetor_imgs[0], vetor_imgs[1], vetor_imgs[2]))
    vetor_imgs.clear()

  vetor_final_imgs_saudaveis = np.array(vetor_final_imgs_saudaveis)  
  print(vetor_final_imgs_saudaveis.shape)

  #Número de imagens que o conjunto de treino terá (60% dos saudáveis e doentes)
  n_imgs_treino_doe = round((len(vetor_final_imgs_doentes)*0.60))
  n_imgs_treino_sau = round((len(vetor_final_imgs_saudaveis)*0.60))
  print(n_imgs_treino_doe)
  print(n_imgs_treino_sau)

  #Número de imagens que o conjunto de teste terá (20% dos saudáveis e doentes)
  n_imgs_test_doe = n_imgs_treino_doe + round((len(vetor_final_imgs_doentes)*0.2))
  n_imgs_test_sau = n_imgs_treino_sau + round((len(vetor_final_imgs_saudaveis)*0.2))
  print(n_imgs_test_doe)
  print(n_imgs_test_sau)


  #Utilizando cerca de 65% das imagens para treino
  treino.extend(vetor_final_imgs_doentes[:n_imgs_treino_doe])
  treino.extend(vetor_final_imgs_saudaveis[:n_imgs_treino_sau])

  #Utilizando cerca de 20% das imagens para teste
  teste.extend(vetor_final_imgs_doentes[n_imgs_treino_doe:n_imgs_test_doe])
  teste.extend(vetor_final_imgs_saudaveis[n_imgs_treino_sau:n_imgs_test_sau])

  #Utilizando cerca de 15% das imagens para validação
  val.extend(vetor_final_imgs_doentes[n_imgs_test_doe:])
  val.extend(vetor_final_imgs_saudaveis[n_imgs_test_sau:])

  treino = np.array(treino)
  teste = np.array(teste)
  val = np.array(val)

  #LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
  for i in range(treino.shape[0]):
    if i < n_imgs_treino_doe:
      labels_treino.append(0)
    else:
      labels_treino.append(1)

  for i in range(teste.shape[0]):
    if i < (n_imgs_test_doe - n_imgs_treino_doe):
      labels_teste.append(0)
    else:
      labels_teste.append(1)

  for i in range(val.shape[0]):
    if i < (len(vetor_final_imgs_doentes) - n_imgs_test_doe):
      labels_val.append(0)
    else:
      labels_val.append(1)
  
  #Shuffling the vectors of images and labels
  
  random.seed(852741)

  random.shuffle(treino)
  random.shuffle(labels_treino)

  random.shuffle(teste)
  random.shuffle(labels_teste)
  
  random.shuffle(val)
  random.shuffle(labels_val)
  


  print(f"Labels de Treino: {labels_treino}")
  print(f"Labels de Teste: {labels_teste}")
  print(f"Labels de Validação: {labels_val}")
  
  labels_treino = np.array(labels_treino)
  labels_teste = np.array(labels_teste)
  labels_val = np.array(labels_val)


  print(treino.shape)
  print(teste.shape)
  print(val.shape)


  
  if model != None:
    classifier(treino, teste, val, labels_treino, labels_teste, labels_val, model)
  else:
    classifier(treino, teste, val, labels_treino, labels_teste, labels_val)
  

def classifier(train_set, test_set, val_set, train_label_set, test_label_set, val_label_set, model = None):

  callback_funcs = [
    tf.keras.callbacks.ModelCheckpoint(filepath = "model.{epoch:02d}-{val_accuracy:.2f}", monitor="val_accuracy", save_best_only=True),
  ]

  if(model == None):
    model = Sequential([
        TCN(nb_filters = 64, kernel_initializer="glorot_uniform", dilations = (1,2,4,8,16,32,64,100,200)),
        tf.keras.layers.Dense(units=1,activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss = tf.losses.BinaryFocalCrossentropy(), metrics = ["accuracy"])
    print("\n\n")
    
    hist = model.fit(train_set, train_label_set, validation_data=(val_set, val_label_set), epochs=200, callbacks = callback_funcs)

    model.summary()

  yhat = model.predict(test_set)

  print("Prediction Shape:")
  evaluated = []
  expected = np.ndarray.tolist(test_label_set)

  for i, val in enumerate(yhat):
    
    if val < .5:
      evaluated.append(0)
      print(f"0    -    {test_label_set[i]}")
    else:
      evaluated.append(1)
      print(f"1    -    {test_label_set[i]}")

  #Salvando o modelo
  if os.path.isdir("/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/last_model") == False:
    model.save("/content/drive/MyDrive/PesquisaNAFLD/Data/NafldClassificationImage/last_model")

  resModel(expected, evaluated)
  

def resModel(expected, evaluated):
    
  truePos=falsePos=falseNeg=trueNeg = 0
  for i, val in enumerate(evaluated):

    #Se o modelo previu a classe da imagem corretamente
    if val == expected[i]:
      #Elencando um verdadeiro positivo
      if val == 0:
        truePos += 1
      #Elencando um verdadeiro negativo
      else:
        trueNeg += 1
    #Se o modelo previu a classe da imagem incorretamente
    else:
      #Elencando um falso positivo
      if val == 0:
        falsePos += 1
      else:
        falseNeg += 1
  print("\n\n\t====== Elucidação dos Resultados ======")
  print(f"Falsos Negativos: {falseNeg}")
  print(f"Falsos Positivos: {falsePos}")
  print(f"Verdadeiros Negativos: {trueNeg}")
  print(f"Verdadeiros Positivos: {truePos}")

  print(f"\nAcurácia: {(truePos + trueNeg)/(truePos + falsePos + trueNeg + falseNeg):.2f}")
  print(f"Sensibilidade: {truePos/(truePos + falseNeg):.2f}")
  print(f"Especificidade: {trueNeg/(falsePos + trueNeg):.2f}")
  print(f"Precisão: {truePos/(truePos + falsePos):.2f}")


    


In [ ]:
#Test with other approach to the problem

def createTimeSeries():

  csv_path = 

  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    print(path_paciente)
    #for i, img in enumerate(os.listdir(os.path.join(path_doentes, paciente))):
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
      if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):


In [ ]:
#Preprocessing Layer

import os, shutil
import csv
import matplotlib as mp
from PIL import Image as img
import numpy as np
from scipy.signal import wiener #modulo para preprocessamento




#Definindo caminhos de onde vou tirar os csvs brutos
abs_path = os.path.abspath(os.getcwd())#pegando o caminho absoluto(raiz)
data_path = os.path.join(abs_path, 'raw_data') # pip install opencv-python-headlesscwd/raw_data
sub = 'flir_export_1' #Escolhendo qual flir usar
sub_path = os.path.join(data_path, sub)  #cwd/raw_data/ flir_export_1(varia de acordo com o escolhido na linha anterior)  
#Definindo caminho de onde vou colocar os csvs
csv_folder = os.path.join(abs_path, 'clean_csv') #cwd/clean_csv
#Definindo caminho de onde vou colcar as imagens
img_path = os.path.join(abs_path, 'images') 





#Renomeia os csvs e os limpa

def clearCsv (sub_path):

    if os.path.exists(csv_folder): 
        shutil.rmtree(csv_folder) #Remove a pasta, caso exista
        pass

    os.mkdir(csv_folder)#cria (ou recria) o diretório para os csvs
    os.mkdir(os.path.join(csv_folder, 'doentes'))#criando subdiretório para doentes
    os.mkdir(os.path.join(csv_folder, 'nao_doentes'))#para não doentes

    for group in os.listdir(sub_path): # Doentes ou não doentes

        group_path = os.path.join(sub_path, group) # cwd/raw_data/flir_export_1/(nao_)doentes
        i = 1 # Numera os pacientes de cada grupo

        for subject in os.listdir(group_path): # Para cada paciente dentro do seu grupo

            subject_path = os.path.join(group_path, subject) # cwd/raw_data/flir_export_1/(nao_)doentes/paciente X

            # Definindo e criando caminho da pasta de destino dos CSVs do paciente atual
            if "nao" in group:
                dest_sheet_path = os.path.join(csv_folder, 'nao_doentes',
                    'nao_doente_{}'.format(i)) # cwd/clean_csv/nao_doentes/nao_doente_X
            else:
                dest_sheet_path = os.path.join(csv_folder,
                    'doentes', 'doente_{}'.format(i)) # cwd/clean_csv/doentes/doente_X
                pass

            os.mkdir(dest_sheet_path)#criando pasta de destino para os csvs dos doentes
            i += 1

            for sheet in os.listdir(subject_path):
                sheet_path = os.path.join(subject_path, sheet) # cwd/raw_data/flir_export_1/(nao_)doentes/paciente X/IR_XXXXX.csv

                relevantRows = 0
                topUselessRows = 0
                relevantCols = 0
            
             # Tratando o CSV
                with open(sheet_path) as sheet_file:
                    sheet_array = csv.reader(sheet_file, delimiter=';')
                    sheet_list = list(sheet_array)

                    # Região hepática FLIR tem 4 linhas inúteis no topo e 1 no fundo
                    # e uma coluna inútil à esquerda
                    relevantRows = len(sheet_list) - 5
                    topUselessRows = len(sheet_list) - relevantRows - 1
                    relevantCols = len(sheet_list[10]) - 1

                # Coletando o CSV sem a informação inútil
                sheet_clear = pd.read_csv(sheet_path, skiprows=topUselessRows, nrows=relevantRows, 
                usecols=[i for i in range(1,relevantCols+1)], header=None, sep=';') 
                
                # Salvando o CSV limpo no padrão ideal para converter em imagem
                sheet_clear.to_csv(os.path.join(dest_sheet_path, sheet), 
                    index=False, header=False)
            





# Controla a geração de imagens térmicas em escala de cinza a partir dos CSVs e as melhora
def generateImages():

    # Definindo o caminho de origem e destino
    source_path = csv_folder # cwd/data/csv

    # Criando os diretórios de destino
    if os.path.exists(img_path):
        shutil.rmtree(img_path)
    os.mkdir(img_path)
    os.mkdir(os.path.join(img_path, 'doentes'))
    os.mkdir(os.path.join(img_path, 'nao_doentes'))

    # Definindo a temperatura máxima e mínima dentre todos os CSVs da base
    tempMax = 0
    tempMin = 40000
    for group in os.listdir(source_path): # Doentes ou não doentes
        group_path = os.path.join(source_path, group) # cwd/source_path/(nao_)doentes
        for subject in os.listdir(group_path): # Pacientes
            subject_path = os.path.join(group_path, subject) # cwd/source_path/(nao_)doentes/(nao_)doente_X
            for sheet in os.listdir(subject_path): # Imagens
                sheet_path = os.path.join(subject_path, sheet) # cwd/source_path/(nao_)doentes/(nao_)doente_X/IR_XXXXX.csv

                sheet_array = pd.read_csv(sheet_path, header=None, decimal=",")
                sheet_array = pd.DataFrame.to_numpy(sheet_array)
                # Armazenando temperaturas sem casas decimais
                localTempMax = int(np.max(sheet_array)*1000)
                localTempMin = int(np.min(sheet_array)*1000)
                if localTempMax > tempMax:
                    tempMax = localTempMax
                if localTempMin < tempMin:
                    tempMin = localTempMin
                #numRows, numCols = sheet_array.shape ##########
                #if numRows > greaterDimension: ##########
                #    imgSquareSize  = numRows ###########
                #if numCols > greaterDimension: #######
                #    imgSquareSize = numCols ##############

    # Gerando as imagens térmicas
    for group in os.listdir(source_path): # Doentes ou não doentes
        group_path = os.path.join(source_path, group) # cwd/source_path/(nao_)doentes
        for subject in os.listdir(group_path): # Pacientes
            subject_path = os.path.join(group_path, subject) # cwd/source_path/(nao_)doentes/(nao_)doente_X
            # Definindo e criando caminho da pasta de destino das imagens do paciente atual
            if "nao" in group:
                dest_image_path = os.path.join(img_path, 'nao_doentes', subject) # cwd/data/images/nao_doentes/nao_doente_X
            else:
                dest_image_path = os.path.join(img_path, 'doentes', subject) # cwd/data/images/doentes/doente_X
            os.mkdir(dest_image_path)
            for sheet in os.listdir(subject_path): # Imagens
                sheet_path = os.path.join(subject_path, sheet) # cwd/source_path/(nao_)doentes/(nao_)doente_X/IR_XXXXX.csv

                # Coletando o CSV
                sheet_array = pd.read_csv(sheet_path, header=None, decimal=",")
                sheet_array = pd.DataFrame.to_numpy(sheet_array)

                # Convertendo o CSV para imagem térmica
                image = toThermography(sheet_array, tempMin, tempMax)

                # Melhorando a imagem
                enhanced_image = enhance(image)

                # Salvando a imagem
                output_image = img.fromarray(np.uint8(enhanced_image))
                output_image.save(os.path.join(dest_image_path, sheet.replace("csv", "jpg"))) # cwd/data/images/(nao_)doentes/(nao_)doente_X/IR_XXXXX.jpg






# Converte uma matriz de temperaturas em uma matriz de valores de cinza, mantendo proporção
def toThermography(temperatureSheet, temperatureMin, temperatureMax):

    # Definindo as escalas para conversão
    grayMax = 255
    grayMin = 0

    # Normalização linear aplicada a todas as células do array para converter
        # de valores de temperatura a valores de pixel
    normalization = lambda i: int((((int(i*1000) - temperatureMin) * (grayMax - grayMin)) / (temperatureMax - temperatureMin)) + grayMin)
    vectorized_normalization = np.vectorize(normalization)
    image = vectorized_normalization(temperatureSheet)

    return image





# Remove ruído e equaliza o histograma de uma matriz de pixels
def enhance(image):


    # Removendo ruído
    #denoised = cv2.medianBlur(image.astype(np.uint8),5)
    #denoised = cv2.GaussianBlur(image.astype(np.uint8),(5,5),0)
    denoised = wiener(image.astype(np.uint8), 5)
    rows, cols = image.shape
    # Corrigindo glitch wiener em 2 pixels de borda
    
    for i in range(rows):
        for j in range(cols):
            if i == 0 or i == 1 or i == len(image)-1 or i == len(image) - 2:
                denoised[i][j] = image[i][j]
            if j == 0 or j == 1 or j == len(image[0])-1 or j == len(image[0])-2:
                denoised[i][j] = image[i][j]
    
    
    # Equalizando histograma
    #denoised_equalizedhist = cv2.equalizeHist(denoised.astype(np.uint8))
    clahe = cv2.createCLAHE()
    denoised_equalizedhist = clahe.apply(denoised.astype(np.uint8))

    return denoised_equalizedhist




# Função de interface
def preprocess(sub_path):
    # Criando a pasta data onde ficarão as imagens e CSVs
    if not os.path.exists(img_path):
        os.mkdir(img_path)

    # Limpando os CSVs
    clearCsv(sub_path)
    # Gerando as imagens a partir dos CSVs
    generateImages()
    print("Boa execução!!!")

    
    

#chamando a função de interface
preprocess(sub_path)




Boa execução!!!


In [ ]:
#Padding Operation into the non-linear size database

def paddingImages():

  for paciente in os.listdir(path_doentes):
    path_paciente = os.path.join(path_doentes, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      
      image = cv2.imread(img_path)
      
      new_height = 283
      new_width = 283
      old_height, old_width, channels = image.shape


      final_image = np.full((new_height, new_width, channels), (0,0,0), dtype = np.uint8)

      x_center = (new_width - old_width) // 2
      y_center = (new_height - old_height) // 2

      final_image[y_center:y_center+old_height, 
       x_center:x_center+old_width] = image

      new_img_name = img_path.split("/")

      os.remove(img_path)

      cv2.imwrite(path_paciente + "/" + new_img_name[-1], final_image)
  
  for paciente in os.listdir(path_saudaveis):
    path_paciente = os.path.join(path_saudaveis, paciente)
    print(path_paciente)
    paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
    for i, img_path in enumerate(paths_imgs):
      
      image = cv2.imread(img_path)
      
      new_height = 283
      new_width = 283
      old_height, old_width, channels = image.shape


      final_image = np.full((new_height, new_width, channels), (0,0,0), dtype = np.uint8)

      x_center = (new_width - old_width) // 2
      y_center = (new_height - old_height) // 2

      final_image[y_center:y_center+old_height, 
       x_center:x_center+old_width] = image

      new_img_name = img_path.split("/")

      os.remove(img_path)

      cv2.imwrite(path_paciente + "/" + new_img_name[-1], final_image)



